# Prisoners Dilemma

First we need imports

In [ ]:
import numpy as np
import time
from ipywidgets import FloatProgress
from IPython.display import display

Then we will need to define the whole bot class. It will work by having a set of parameters that take percentage values, and their various combinations define the multiple various strategies a bot in the game #prisoners dilemma# can take. Most common strategies can be implemented through this method. A few examples will be detailed below.

In [ ]:
class bot(object):
    def __init__(self,params):
        # Bot behavior parameters
        self.PC = params["PC"]
        self.PCorg = params["PC"]
        self.MS = params["MS"]
        self.MD = params["MD"]
        self.LS = params["LS"]
        self.LD = params["LD"]
        self.R  = params["R"]
        # Record keeping variables
        self.score = 0 # Average "money" made
        self.wins = 0 # Average number of wins (rounds)
        self.games_played = 0 # Rounds played
        
        self.name = params["name"]
        
    def getScore(self): # Get data
        return self.score, self.wins
    
    def getParams(self):
        return {"PC":self.PCorg,
                "MS":self.MS,
                "MD":self.MD,
                "LS":self.LS,
                "LD":self.LD,
                "R":self.R}
    
    def getName(self):
        return self.name
    
    def updateScore(self,result,win): # Update the bots personal records like score and wins
        self.score = ((self.score * self.games_played) + result)/(self.games_played +1)
        self.wins = ((self.wins * self.games_played) + win)/(self.games_played +1)
        self.games_played = self.games_played + 1
        
    def play(self,c): # Implement strategy, c is the other bots previous choice
        M=self.MS if c==1 else self.MD
        L=self.LS if c==1 else self.LD
        self.PC = self.PC - (L*(self.PC - M))
        
    def strategy(self,c=None): # The bots strategy.
        if c: # if not first round
            self.play(c) # implement strategy
        else:
            self.PC = self.PCorg
        
        if (np.random.rand() < self.PC) == (np.random.rand() > self.R): # Implement the randomness in the choice.
            return 0 # Steal
        else:
            return 1 # Deal

Now we create a small function that creates random parameters for bots to use.

In [ ]:
def randParams():
    params = {"PC":np.random.rand()*100,
              "MS":np.random.rand()*100,
              "MD":np.random.rand()*100,
              "LS":np.random.rand()*100,
              "LD":np.random.rand()*100,
              "R" :np.random.rand()*100,}
    params.update({"name":"{}-{}-{}-{}-{}-{}".format(int(params["PC"]),int(params["MS"]),int(params["MD"]),int(params["LS"]),int(params["LD"]),int(params["R"]))})
    return params

Now we need to make the little simulator engine that takes two random bots, and lets them battle against each other.

In [ ]:
def game(a,b,r,payoff): # a = bot a, b = bot b, r = number of rounds
    ca=None
    cb=None
    for i in range(r):
        ca = a.strategy(ca)
        cb = b.strategy(cb)
        w = 0.5 if cb==ca else ca<cb
        a.updateScore(payoff[ca][cb][0],w)
        b.updateScore(payoff[ca][cb][1],1-w)

Now we need to make a little function that selects bots

In [ ]:
def select(botlist):
    return np.random.choice(botlist,2,replace=False)

Finally, we just need a function that spits out our results. (This function can be expanded upon later)

In [ ]:
def printResults(botlist, q=None):
    winlist = [i.getScore()[1] for i in botlist]
    scorelist = [i.getScore()[0] for i in botlist]
    names = [i.getName() for i in botlist]
    
    wsort = np.argsort(np.array(winlist)*-1)
    ssort = np.argsort(np.array(scorelist)*-1)
    
    t = len(botlist) if len(botlist) < 10 else 10
    
    print("Ranked results ordered by average wins")
    print("Rank AvgWins AvgScore BotName")
    for i in range(t):
        print(" {:2d}    {:.2f}    {:.2f}   {}".format(i+1,winlist[wsort[i]],scorelist[wsort[i]],names[wsort[i]]))
    print("Ranked results ordered by average score")  
    print("Rank AvgScore AvgWins BotName")
    for i in range(t):
        print(" {:2d}    {:.2f}    {:.2f}   {}".format(i+1,winlist[ssort[i]],scorelist[ssort[i]],names[ssort[i]]))
        
    if q:
        print("********************")
        print("RESULT OF YOUR BOT:")  
        print("********************")
        print("RankWins AvgWins RankScores AvgScores BotName")
        print(" {:2d}        {:.2f}      {}         {:.2f}   {}".format(np.argmax(wsort)+1,winlist[-1],np.argmax(ssort)+1,scorelist[-1],names[-1]))

And we might want some sort of progress bar to get some feeling of how quick this is. Not actually used atm.

In [ ]:
def prog(a,b):
    bar_width = 40
    c = bar_width * a / b
    l = bar_width - c
    print('\r' + u"\u2588" * int(c) + '-' * int(l))

## Now we can prep everything, but first we need some basic inputs

We need to know the following: how many number of bots in the original batch (n), how many rounds to play in a matchup (r), how many matchups (m), and the payoff matrix (pom). Fill in the numbers below, then press ***run cell*** or ***ctrl + enter***
Note about payoff matrices. They work like this: each player has x number of actions, here steal or deal. That makes a 2x2 matrix. In each cell, there are the payoffs for bot 1 and bot 2, respectively, if the bots choose the corresponding actions. In the example below, if both steal, both get 1 score each. If bot B steals, and bot A deals, bot B gets 5, and bot A gets 0, and so forth.

In [ ]:
n = 300 # number of bots
r = 90 # rounds to play
m = 2000 # number of matchups to start with
mp = 500 # number of matchups for custom bots later

 # BOT B - STEAL - DEAL
pom =    [[[1,1] , [5,0]], # BOT A STEAL
          [[0,5] , [3,3]]] # BOT A DEAL  

If you remembered to do ***run cell*** / ***ctrl + enter*** on the previous cell (you can check that by seeing that the ln [x]: updates by one each time you do it), we can now do the same on the next section. This might take some time depending on what values you used in the previous cell.

In [ ]:
botlist = [bot(randParams()) for i in range(n)] # Creating the botlist
f = FloatProgress(min=0, max=m) # Progress bar
display(f)
for i in range(m):
    sel = select(botlist)
    game(sel[0],sel[1],r,pom)
    f.value = i

Now everything should be ready for FUN TIMES!

# Now let's make bots, add them to the list, and let them compete against the field.

### Note: each bot added will be in the field for later as well.

What you need to do is, fill in the values you want, then ***run cell*** / ***ctrl + enter***, and results should pop up with the position and score of your bot at the top of the output.

In [ ]:
# Add your bot params here! Everything is in percentages, and unless otherwise noted, it's P(steal).
params = {"PC": 100.0,   # Starting probability for STEAL
          "MS": 70.0,  # Target probability for stealing, IF opponent STEALS all the time
          "LS": 0.0,  # How quick PC goes to MS if opponent steals all the time, 0 = never, 100.00 = instantly
          "MD": 20.0,  # Target probability for stealing, IF opponent DEALS all the time
          "LD": 0.0,  # How quick PC goes to MD if opponent deals all the time, 0 = never, 100.00 = instantly
          "R" : 0.0,   # How often the bot does the opposite of what it usually would do
          "name": "Stealer" # This is the name. Normal string rules apply
         }
a = bot(params) # Creating your bot!

f = FloatProgress(min=0, max=mp) # Progress bar
display(f)
for i in range(mp): # Letting your bot compete against a random selection of bots!
    game(a,select(botlist)[0],r,pom)
    f.value = i

botlist.extend([a]) # Adding the bot to the list now that it's been initiated into the fun club
print("AND HERE BE THE RANKED RESULTS!")
printResults(botlist,1)